# PyLossless pipeline: A step-by-step demonstration

This Notebook details each pipeline step. First, the mathematical notation for the operation will be described. Second, the python code for the operation will be shown. We will go through the first step in details. Similar procedures are used in all following steps so for the sake of brevity we will increasingly explain less of the formulas throughout this tutorial

# ----------------------------------------------------------------------------

# Cloning and installing pylossless

In [ ]:
# THIS CODE ONLY NEEDS TO BE RUN ONCE.
# IF PYLOSSLESS IS ALREADY AVAILABLE, IT DOES NOT NEED TO BE RERUN.
try:
  import pylossless
except ModuleNotFoundError:
  !git clone https://github.com/lina-usc/pylossless.git
  %pip install --quiet --editable ./pylossless

  # RESTARTING THE KERNEL AFTER THE PIP INSTALL
  # THIS IS NECESSARY FOR THE NEWLY INSTALLED PACKAGE
  # TO BE CORRECTLY IMPORTED.
  exit()


<br>

--------------------------------
# Getting a LosslessPipeline object

 Create a pipeline object and manually set a raw object to it. We proceed this way to be able to demonstrate the operation for each step, one at a time. Normally, one would simply load and process a raw object with `pipeline.run_with_raw(raw)`. This demonstration relies on test datasets from MNE. It may take a while to download the MNE sample data (about 2-3 minutes on Google Colab). **Please be patient**...

In [ ]:
from pathlib import Path

import numpy as np

import mne
from mne.datasets import sample

import pylossless as ll

# LOAD DATA FROM MNE FOR THE DEMONSTRATION
data_path = sample.data_path()
meg_path = data_path / 'MEG' / 'sample'
raw_fname = meg_path / 'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(raw_fname, preload=True)

# LOAD DEFAULT CONFIG
config = ll.config.Config()
config.load_default()
config.save("test_config.yaml")

# CREATE A PIPELINE AN MANUALLY SET A RAW OBJECT
pipeline = ll.LosslessPipeline('test_config.yaml')
pipeline.raw = raw

--------------------------

# Preamble: Notation

We define:
- s, e, and t, as sensor, epochs, and time dimensions, respectively.

- a 3D matrix $X \in \mathbb{R}^{S_\mathcal{G} \times E_\mathcal{G} \times T}$, where $S_\mathcal{G}$  and $E_\mathcal{G}$ are the set of good sensors and epochs respectively, and $T$ is the number of time points (i.e. samples)

- We use superscript to denote operations across a dimension, and we use subscript to denote indexing a dimension


- a single sensor $i$ as $ X\big|_{s=i} \in \mathbb{R}^{E_\mathcal{G} \times T}$, with $i \in S_\mathcal{G}$

- a single epoch $j$ as  $X\big|_{e=j} \in \mathbb{R}^{S_\mathcal{G} \times T}$, with $j \in E_\mathcal{G} $

- sensor specific thresholds for rejecting epochs as $\tau^e_i \in \mathbb{R}^{S_\mathcal{G}}$

- epoch specific thresholds for rejecting sensors as $\tau^s_j \in \mathbb{R}^{E_\mathcal{G}}$


- *quantiles* as $Q\#^{dim}$: i.e. $Q75^s$ is the 75th *quantile* along the sensor dimension. The function $Q75^s(X)$ computes the 75th quantile along the $s$ dimension of matrix $X$, resulting in a matrix noted $X^{Q75^s} \in \mathbb{R}^{E \times T}$

Throughout the text, we use capital letters for matrices and lower-case letters for scalars. For example, the data point for sensor $i$, epoch $j$ and time $k$ as $X\big|_{s=i; e=j; t=k} = x_{ijk}\in \mathbb{R}$, and $X=\{x_{ijk}\}$.

<br>

------------------------

# Step 0: Input Data

### First, we epoch the data to be used for step 1

Let our 3D matrix below be defined as $X \in \mathbb{R}^{S \times E \times T}$ where $X$ is a matrix of real numbers and of dimension $S$ sensors $\times$ $E$ epochs $\times$ $T$ times.

In [ ]:
epochs = pipeline.get_epochs()

In [ ]:
# Let us convert our epochs array into a Named Array
from pylossless.pipeline import epochs_to_xr
epochs_xr = epochs_to_xr(epochs, kind='ch')
epochs_xr # 277 epochs, 50 sensors, 602 samples per epoch

-------------------------

# Step 1: Flag Noisy Sensors

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_noisy_sensors.png" alt="Flag noisy channels" width="50%" height="50%">


## step 1a) Take standard deviation across temporal dimension

#### We take the standard deviation of $X \in \mathbb{R}^{S \times E \times T}$ across the samples dimension time, resulting in a 2D matrix $X^{\sigma_{t}} \in \mathbb{R}^{S \times E}$


![std across samples](https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/samples_std.png)


<br>

---------------------------------


In [ ]:
# Take standard deviation across time dimension
trim_ch_sd = epochs_xr.std('time')
# display a small subset
trim_ch_sd

------------------

## Take the 50th and 75th quantile across dimension sensor of $X^{\sigma_{t}}$


This operation results in two 1D vectors of size $E$:

- $ X^{{\sigma}_t{Q50^s}} = Q50^s(X^{\sigma_{t}}) \in \mathbb{R}^{E} $
- $ X^{{\sigma}_t{Q75^s}} = Q75^s(X^{\sigma_{t}}) \in \mathbb{R}^{E} $



In [ ]:
# Step a
q50, q75 = trim_ch_sd.quantile([.5, .75], dim='ch')
q50 # a 1D array of median stDev values across channels for each epoch

## 1b) Let the *Upper Quantile Range* be defined by $Q75 - Q50$
$ UQR^s = X^{{\sigma}_T{Q75}^s} - X^{{\sigma}_T{Q50}^s}$

In [ ]:
# step B. Define the threshold
uqr = q75 - q50
uqr  # Q70 - Q50

---------------

### Identify outlier Indices $(i, j)$

We multiply a constant $k$ by the $UQR$ to define a measure for the spread of the right tail of the distribution of $X^{\sigma_{t}}$ values and add it to the median of $X^{\sigma_{t}}$ to obtain epoch-specific standard deviation threshold for outliers:

$$
    \tau^s_j = X^{{\sigma}_T{Q50}^S} + UQR^s\times k
$$

That is, $\tau^s_j$ is the epoch-specific threshold for the epoch $j$.

In [ ]:
# Step 1b part 2
k = 3
u_out =  q50 + q75 * k
u_out  # epoch specific thresholds

------------

Now, we compare our 2D standard deviation matrix to the threshold vector of $\tau^e_j$:

$$
X^{\sigma_{t}} \big|_{e=j}  > \tau^s_j
$$

resulting in the indicator matrix $C \in \{0, 1\}^{S \times E}=\{c_{ij}\}$ with

$$
c_{ij} =
\begin{cases}
0 & \text{if } x^{\sigma_{t}}_{ij} < \tau^s_j \\
1 & \text{if } x^{\sigma_{t}}_{ij} \geq \tau^s_j
\end{cases}
$$

Each element of this matrix indicates whether sensor $i$ is an outlier at an epoch $j$.


In [ ]:
# Step D
outlier_mask = trim_ch_sd > u_out
outlier_mask


--------------

## 1c) Identify noisy sensors part 1
To identify outlier **sensors**, we average across the **epoch** dimension of our indicator matrix $C$ and obtain $C^{\mu_e} \in \mathbb{R}^{S_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_e}_i$ representing the percentage of **epochs** for which that sensor is an outlier.


In [ ]:
prop_outliers = outlier_mask.astype(float).mean('epoch')
prop_outliers # porportion of epochs that sensor is an outlier

## 1d) Identify noisy sensors part 2

Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a sensor should be marked artifactual. The sensor $i$ is flagged as noisy if $c^{\mu_e}_i > \tau^{p}$.


In [ ]:
threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED SENSORS

In [ ]:
# Let's add the outlier channels to our flags
bad_chs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED SENSORS
pipeline.flags["ch"].add_flag_cat(kind='ch_sd',
                                  bad_ch_names=bad_chs)
print(pipeline.flags['ch'])

---------------

# Step 2: Flag Noisy Epochs

#### This step closely resembles the "Flag Noisy Channels" step. For the sake of brevity we will be more concise in the documentation.

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_noisy_epochs.png" alt="Flag noisy epochs" width="50%" height="50%">

--------



----------------------------------------------------------------------------


## step 2a) standard deviation across the samples dimension $T$


### **Take a moment below to notice** that the sensors flagged in step 1 are not in ``epochs_xr``


In [ ]:
epochs = pipeline.get_epochs()
# Let's make our epochs array into a named Array
epochs_xr = epochs_to_xr(epochs, kind='ch')
trim_ch_sd = epochs_xr.std("time")
trim_ch_sd

----------

## Step 2a) part 2: We compute $\sigma^{X^{\sigma_{t}}[50,70]e}$




 50th and 75th quantile across epochs and the UQR

<br>

Like before, We Take the median and 70th quantile, but now we operate across **epochs**, resulting in two 1D vector's of size ``n_good_sensors`` $S_\mathcal{G}$

<br>

- $ X^{{\sigma}_t{Q50^e}} = Q50^e(X^{\sigma_{t}}) \in \mathbb{R}^{S_\mathcal{G}} $
- $ X^{{\sigma}_t{Q75^e}} = Q75^e(X^{\sigma_{t}}) \in \mathbb{R}^{S_\mathcal{G}} $

<br>

$ UQR^e = (X^{{\sigma}_T{Q75}^e} - X^{{\sigma}_T{Q50}^e})$

<br>

--------------------------------

## Step 2b: define sensor-specific thresholds for rejecting epochs $\tau^e_i$

Our sensor-specifc threshold for rejecting epochs is defined by:


$ \tau^e_i = X^{{\sigma}_T{Q50}^e} + UQR^e\times k$



In [ ]:
# This is step 2a
mid_val, upper_val = trim_ch_sd.quantile([.5, .75], dim='epoch')
upper_dist = upper_val - mid_val
upper_dist

In [ ]:
# This is step 2b
k = 3
u_out =  mid_val + upper_dist * k # sensor-specific thresholds

--------

## Step 2c) Identify Noisy Epochs part 1

<br>

#### The indicator matrix is defined by

$$
c_{ij} =
\begin{cases}
0 & \text{if } x^{\sigma_{t}}_{ij} < \tau^e_i \\
1 & \text{if } x^{\sigma_{t}}_{ij} \geq \tau^e_i
\end{cases}
$$

To identify outlier **epochs**, we average across the **sensor** dimension of our indicator matrix $C$ and obtain $C^{\mu_s} \in \mathbb{R}^{E_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_s}_j$ representing the percentage of **sensors** for which that epoch is an outlier.


In [ ]:
# This is indicator matrix C
outlier_mask = trim_ch_sd > u_out
outlier_mask

In [ ]:
# This is Step 2c
prop_outliers = outlier_mask.astype(float).mean('ch')
prop_outliers = prop_outliers.drop_vars('quantile')
prop_outliers  # percent of sensors that are bad during each epoch

----

## Step 2d: Identify Noisy Epochs Part 2

Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a epoch should be marked artifactual. The epoch $j$ is flagged as noisy if $c^{\mu_s}_j > \tau^{p}$.


In [ ]:
threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED EPOCHS

In [ ]:
# Let's add the outlier epochs to our flags
bad_epochs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED Epochs
pipeline.flags["epoch"].add_flag_cat(kind='ch_sd',
                                  bad_epoch_inds=bad_epochs,
                                     epochs=epochs)
print(pipeline.flags['epoch'])

------


<br>
<br>


# **Step 3**: Find Nearest Neighbours & return Maximum Correlation


*Wheras steps 1 and 2 operated on a 2D matrix of standard deviation values, Steps 5, 6, 7, and 8 will operate on correlation coefficients. This Step describes the procedure for defining the 2D matrix of correlation coefficients.*

<br>

![Get nearest neighbours](https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Nearest_neighbors.png)


-----

In [ ]:
from pylossless.pipeline import chan_neighbour_r
# Posting full function definition for easier inspection
chan_neighbour_r??

<br>

#### **Take a moment to notice** that the flagged epochs from the previous step are dropped during epoching.

<br>

In [ ]:
# Notice that our flagged epochs are dropped.
epochs = pipeline.get_epochs()


----------------------



## Step 3, part 1: Calculate Correlation Coefficents between each Sensor and it's nearest Neighbors

<br>

- For each good sensor $i$ in $S_{\mathcal{G}}$, we select its $N$ nearest neighbors. I.e. the $N$ sensors that are closest to it.

- We call the sensor $i$ the *origin*, and its nearest neighbors $\hat{s_l}$ with $l \in \{1, 2, \ldots, N\}$

- Then, for each epoch $j$, we calculate the correlation coefficient $\rho^t_{(i,\hat{s_l}),j}$ between origin sensor $i$ and each neighbor $\hat{s_l}$ across dimension $t$ (samples):

Returning a 3D matrice of correlation coefficients

$$
\mathrm{P}^t = \{\rho^t_{(i, \hat{s_l}),j}\} \in \mathbb{R}^{S_G \times E_G \times n}
$$

Finally, we select the maximum correlation coefficient across the neighbor dimension $n$:


$$
\mathrm{P}^{t,{\text{max}}^n}= \max\limits_{\hat{s_l}}  \rho^t_{(i, \hat{s_l}),j}
$$


Returning a 2D matrix where each value at $(i, j)$ is the maximum correlation coefficient between sensor $i$ and its $N$ nearest neighbors, for epoch $j$

------------------------

In [ ]:
data_r_ch = chan_neighbour_r(epochs, nneigbr=3, method='max')
# maximum correlation out of correlations between ch and its 3 neighbors
data_r_ch


---------------------

<br>

### This matrix $\mathrm{P}^{t,{\text{max}}^n}$  will be used in steps 5, 6, 7, and 8 below.

<br>

-------------------

# **Step 5**: Flag Bridged Sensors

<br>


<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_bridged_sensors.png" width="50%" />



<br>

---------------------

## <center>Operations<center>

<br>

### 6a) Take the 25th, 50th, and 75th quantile across the epochs dimension of $\mathrm{P}^{t,{\text{max}}^n}$

#### *and calculate the Inter Quantile Range (IQR)*


$$
IQR^e = \mathrm{P}^{t,{\text{max}}^nQ75^e} - \mathrm{P}^{t,{\text{max}}^nQ25^e}
$$

For each sensor, divide the median across epochs by the IQR across epochs. bridged channels should have a high median correlation but a low IQR of the correlation. We call this measure the bridge-indicator.

$$
\mathcal{B}_s = \frac{\mathrm{P}^{t,{\text{max}}^nQ50^e}}{IQR^e}
$$

<br>

-------------------------------



### 6b) Define a bridging threshold

Take the 25th, 50th, and 75th quantile of $\mathcal{B}_s$

And calculate the Inter Quantile Range* $IQR^s$. A channel $i$ is bridged if $\mathcal{B}_i > B^{Q50^s} +k \times IQR^s$.

In [ ]:
import scipy
from functools import partial

msr = data_r_ch.median("epoch") / data_r_ch.reduce(scipy.stats.iqr, dim="epoch")
# msr is a 1D vector of size n_sensors
config_trim = 40
config_bridge_z = 6

trim = config_trim
if trim >= 1:
        trim /= 100
trim /= 2 # .20 and will be used as (.20, .20)

trim_mean = partial(scipy.stats.mstats.trimmed_mean,
                        limits=(trim, trim))
trim_std = partial(scipy.stats.mstats.trimmed_std,
                        limits=(trim, trim))

z_val = config_bridge_z  # 6
mask = (msr > msr.reduce(trim_mean, dim="ch")
        + z_val*msr.reduce(trim_std, dim="ch")) # bridged chans

bridged_ch_names = data_r_ch.ch.values[mask]
bridged_ch_names

In [ ]:
# Let's add the outlier channels to our flags
bad_chs = bridged_ch_names
pipeline.flags["ch"].add_flag_cat(kind='bridge',
                                  bad_ch_names=bad_chs)
pipeline.flags['ch']

# Step 6: Flag Rank Channel

## The rank sensor $i$ is simply the sensor with the highest median (across epochs) correlation coefficient, out of the remaining set of good sensors, i.e.,

$$
i = \text{arg}\max\limits_i \rho_{j}^{t,{\text{max}}^n,median^j}
$$

In [ ]:
good_chs = [ch for ch in data_r_ch.ch.values
          if ch not in pipeline.flags["ch"].get_flagged()]
data_r_ch_good = data_r_ch.sel(ch=good_chs)

flag_ch = [str(data_r_ch_good.median("epoch")
                                     .idxmax(dim="ch")
                                     .to_numpy()
                            )]
pipeline.flags['ch'].add_flag_cat(kind='rank', bad_ch_names=flag_ch)
pipeline.flags['ch']['rank']


--------------------------------------

# Step 7: Flag low correlation Channels

- This step closely follows Step 1 except we use the lower end of the distribution to set the outliers threshold.

- We will be succinct in the documentation. Please Feel free to inspect the variables in the python code as needed:


----------------

## <center>Operations<center>

<br>

### 6a) Take the 30th and 50th quantile of $\mathrm{P}^{t,{\text{max}}^n}$ across the sensors dimension

*and calculate the Lower Quantile Range* $LQR^s$

Resulting in vectors $\mathrm{P}^{t,{\text{max}}^nQ25^s}$ and $\mathrm{P}^{t,{\text{max}}^nQ50^s}$ of size $E_\mathcal{G}$. The lower quantile range ($LQR$) is defined as:


$$
LQR^s = \mathrm{P}^{t,{\text{max}}^nQ50^s} - \mathrm{P}^{t,{\text{max}}^nQ25^s}
$$

<br>

### 6b) Define epoch-specific thresholds for rejecting sensors

$\tau^s_j$ represents our set of epoch-specific thresholds:


$$
\tau^s = \mathrm{P}^{t,{\text{max}}^nQ50^s} - LQR^s\times k
$$

Now, we compare our 2D correlation coefficient matrix $\mathrm{P}^{t,{\text{max}}^n} = \{ \rho^{t,{\text{max}}^n}_{ij} \}$ to the threshold vector $\tau^s_j$ resulting in the indicator matrix $C \in \{0, 1\}^{S \times E}=\{c_{ij}\}$ with

$$
c_{ij} =
\begin{cases}
1 & \text{if } \rho^{t,{\text{max}}^n}_{ij} \leq \tau^s_j \\
0 & \text{if } \rho^{t,{\text{max}}^n}_{ij} > \tau^s_j
\end{cases}
$$



### 6c) Identify uncorrelated sensors

To identify uncorrelated **sensors**, we average across the **epoch** dimension of our indicator matrix $C$ and obtain $C^{\mu_e} \in \mathbb{R}^{S_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_e}_i$ representing the percentage of **epochs** for which that sensor is an outlier.


Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a sensor should be marked artifactual. The sensor $i$ is flagged as noisy if $c^{\mu_e}_i > \tau^{p}$.

In [ ]:
# Step a
lower_val, mid_val = data_r_ch.quantile([.3, .5], dim='ch')

# step B. Define the threshold
lower_dist = mid_val - lower_val

# Step B
k = 3
l_out =  mid_val - lower_dist * k

# Step D
outlier_mask = data_r_ch < l_out

prop_outliers = outlier_mask.astype(float).mean('epoch')
prop_outliers = prop_outliers.drop_vars('quantile')

threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED SENSORS

# Let's add the outlier channels to our flags
bad_chs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED SENSORS
pipeline.flags["ch"].add_flag_cat(kind='uncorrelated',
                                  bad_ch_names=bad_chs)
print(pipeline.flags['ch'])


----------------------------

<br>

# Step 8: Flag Uncorrelated Epochs

This step is similar to step 6 (flag uncorrelated channels) and step 2 (flag noisy epochs). For the sake of brevity we will not re-describe each step


---------------------------------

## <center>Operations<center>

<br>

### 6a) Take $LQR^e$ of $\mathrm{P}^{t,{\text{max}}^n}$


### 6b) Defined sensor-specific thresholds

We get the indicator matrix as described previously, using

$$
 \tau^e_i = \mathrm{P}^{t,{\text{max}}^nQ50^e} - LQR^e\times k
$$

and

$$
c_{ij} =
\begin{cases}
1 & \text{if } \rho^{t,{\text{max}}^n}_{ij} < \tau^e_i \\
0 & \text{if } \rho^{t,{\text{max}}^n}_{ij} \geq \tau^e_i
\end{cases}
$$


<br>

### 6c) Identify uncorrelated sensors
<br>

We define a fractional threshold as we did in the previous step and flag uncorrelated epochs $j$ if $c^{\mu_s}_j > \tau^{p}$.



In [ ]:
# Step a
lower_val, mid_val = data_r_ch.quantile([.3, .5], dim='epoch')

# step B. Define the threshold
lower_dist = mid_val - lower_val

# Step B
k = 3
l_out =  mid_val - lower_dist * k

# Step D
outlier_mask = data_r_ch < l_out

prop_outliers = outlier_mask.astype(float).mean('ch')
prop_outliers = prop_outliers.drop_vars('quantile')

threshold = 0.2
prop_outliers[prop_outliers < threshold] # FLAGGED EPOCHS

# Let's add the outlier epochs to our flags
bad_epochs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED EPOCHS
pipeline.flags["epoch"].add_flag_cat(kind='uncorrelated',
                                  bad_epoch_inds=bad_epochs,
                                     epochs=epochs)
print(pipeline.flags['ch'])

# Step 9: Run Initial ICA

In [ ]:
pipeline.run_ica('run1')

# Step 10: Identify outlying ICA Time courses

This step follows exactly step 1 so we will not re-describe the operations here.

There are however a few key differences:

- The input 3D matrix is ``(n_components, n_good_epochs, n_times)``: $ X \in \mathbb{R}^{C\times E_G \times{T}} $
- the default threshold $k$ is 6

In [ ]:
pipeline.flags["ic"]

# Step 11: Run Final ICA

# This is the Final ICA, excluding all flagged epochs and channels.

# The ``ICLabel`` Neural Network will be run on this ICA

In [ ]:
pipeline.run_ica('run2')

# Appendix: Find outlying channels to leave out of Re Reference

- This robust average reference is performed before the first pipeline step and between most pipeline steps.

- It does not flag sensors but identifies outlier sensors and leaves them out of the reference

Like steps 1 and 2, this step starts with 3D matrix $X \in \mathbb{R}^{S \times E \times T}$ and takes the standard deviation across axis $T$:

<br>

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/robust_rereference.png" alt="Find outliers and leave out of rereference" width="50%" height="50%">


<br>

---------------------

## <center>Operations<center>

<br>

### 6a) Take the median, 30th, and 70th quantile across dimension channels of  $X^{\sigma_{t}} \in \mathbb{R}^{S \times E}$, and Calculate the Inter Quantile Range (IQR).

<br>


Create a non-parametric Z score

$$
\mathcal{Z}^{\sigma^{t}} = \frac{X^{\sigma^{t}Q50^s} - X^{\sigma^{t}}}{IQR^s}
$$




<br>

-------------------------------

<br>


### 6b) Define  threshold

Sensor $i$ is left out of the re-reference if

$$
\mathcal{Z}^{\sigma^{t}\mu^{e}} > \mathcal{Z}^{\sigma^{t}\mu^{e}Q50^s} + k \times IQR^s
$$

with $IQR^s = \mathcal{Z}^{\sigma^{t}\mu^{e}Q75^s}-\mathcal{Z}^{\sigma^{t}\mu^{e}Q25^s}$ and where $\mathcal{Z}^{\sigma^{t}\mu^{e}}$ is the mean of $\mathcal{Z}^{\sigma^{t}}$ across epochs.

In [ ]:
# This is step 2a
ch_dist = trim_ch_sd - trim_ch_sd.median(dim="ch")
perc_30 = trim_ch_sd.quantile(0.3, dim="ch")
perc_70 = trim_ch_sd.quantile(0.7, dim="ch")

# This is step b: non-parametric z-score
ch_dist /= perc_70 - perc_30  # shape (chans, epoch)

# This is step c
mean_ch_dist = ch_dist.mean(dim="epoch")

# Step D begins here
# find the median and 30 and 70 percentiles
# of the mean of the channel distributions
mdn = np.median(mean_ch_dist)
deviation = np.diff(np.quantile(mean_ch_dist, [0.3, 0.7]))
print(f"median: {mdn}, std: {deviation}")

# Thresholding. True sensors are left out
mean_ch_dist > mdn+6*deviation

In [ ]:
# Here are any sensor names that are excluded
mean_ch_dist.ch[mean_ch_dist > mdn+6*deviation].values.tolist()